In [317]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import time
import csv

In [496]:
def level_1_predictor(t_train,y, model):
    leng = int(len(t_train)/2)
    t_a, t_b= t_train[:leng], t_train[leng:]
    tg_a, tg_b = y[:leng], y[leng:]
    if model == LogisticRegression:
        m1 = model(max_iter = 20**9)
        m2 = model(max_iter = 20**9)
    else:
        m1 = model()
        m2 = model()
    m1.fit(t_a,tg_a)
    tp_b = [i[1] for i in m1.predict_proba(t_b)]
    m2.fit(t_b,tg_b)
    tp_a = [i[1] for i in m2.predict_proba(t_a)]
    
    tp = [[x] for x in np.hstack((tp_a,tp_b))]
    
    return tp

In [494]:
# I use the best prediction from one of the models to stack.
def training_stacker(feature_sets):
    column_name = list(feature_sets.keys())
    itr = 0
    dic_out = {}
    for y in column_name:
        stacking_feature = np.array(feature[feature_sets[y]])
        print('--',column_name[itr], end= ' ')
        score_set = {}
        for x in models:
            l1_pred = level_1_predictor(stacking_feature,np_label.T[0], x)
            score = roc_auc_score(np_label,l1_pred)
            model_name = x.__name__
            score_set[str(score)]=model_name
        model_for_stacking = score_set[str(max([float(x) for x in score_set.keys()]))]
        print(model_for_stacking)
        dic_out[y]=[model_for_stacking]
        itr+=1
    temp = pd.DataFrame(dic_out)
    model_for_stacking_file = pd.read_csv('model_for_stacking_l1.csv')
    model_for_stacking_file = pd.concat((model_for_stacking_file,temp), axis = 0)
    model_for_stacking_file.to_csv('model_for_stacking_l1.csv', index = False)
#GGLGRGLLLLLL
#GGLGRGLLLLGL

In [ ]:
def updata_important_l1():
    i = pd.read_csv('important_l1.csv')
    for x in important_l1:
        i[x]+=1
    i.to_csv('important_l1')

### 1. get data

In [510]:
df = pd.read_csv(r'C:\Users\johnk\OneDrive\Desktop\project\python project\kaggle\amex-default-prediction\new_train\train_2.csv')

In [511]:
df.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8269 entries, 0 to 8268
Data columns (total 1304 columns):
 #     Column       Non-Null Count  Dtype  
---    ------       --------------  -----  
 0     customer_ID  8269 non-null   object 
 1     P_2_mean     8220 non-null   float64
 2     D_39_mean    8269 non-null   float64
 3     B_1_mean     8269 non-null   float64
 4     B_2_mean     8268 non-null   float64
 5     R_1_mean     8269 non-null   float64
 6     S_3_mean     7214 non-null   float64
 7     D_41_mean    8268 non-null   float64
 8     B_3_mean     8268 non-null   float64
 9     D_42_mean    2027 non-null   float64
 10    D_43_mean    6369 non-null   float64
 11    D_44_mean    7957 non-null   float64
 12    B_4_mean     8269 non-null   float64
 13    D_45_mean    8268 non-null   float64
 14    B_5_mean     8269 non-null   float64
 15    R_2_mean     8269 non-null   float64
 16    D_46_mean    6812 non-null   float64
 17    D_47_mean    8269 non-null   float64
 18    D_48

base_line = label.loc[label.target==0].shape[0]/label.shape[0]

### 2. create feature and label

In [463]:
feature = df.iloc[:,1:-1]
label = pd.DataFrame(df['target'], columns=['target'])
np_label = np.array(label)
feature_name = feature.columns

In [493]:
feature_name

Index(['P_2_mean', 'D_39_mean', 'B_1_mean', 'B_2_mean', 'R_1_mean', 'S_3_mean',
       'D_41_mean', 'B_3_mean', 'D_42_mean', 'D_43_mean',
       ...
       'D_136_max', 'D_137_max', 'D_138_max', 'D_139_max', 'D_140_max',
       'D_141_max', 'D_142_max', 'D_143_max', 'D_144_max', 'D_145_max'],
      dtype='object', length=1302)

In [507]:
important_l1.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8298 entries, 0 to 8297
Data columns (total 225 columns):
 #    Column      Non-Null Count  Dtype  
---   ------      --------------  -----  
 0    D_39_mean   8298 non-null   float64
 1    R_1_mean    8298 non-null   float64
 2    D_42_mean   8298 non-null   float64
 3    D_43_mean   8298 non-null   float64
 4    B_5_mean    8298 non-null   float64
 5    D_46_mean   8298 non-null   float64
 6    B_7_mean    8298 non-null   float64
 7    B_8_mean    8298 non-null   float64
 8    R_3_mean    8298 non-null   float64
 9    P_3_mean    8298 non-null   float64
 10   B_11_mean   8298 non-null   float64
 11   D_54_mean   8298 non-null   float64
 12   S_7_mean    8298 non-null   float64
 13   S_8_mean    8298 non-null   float64
 14   S_9_mean    8298 non-null   float64
 15   B_15_mean   8298 non-null   float64
 16   D_65_mean   8298 non-null   float64
 17   B_18_mean   8298 non-null   float64
 18   D_68_mean   8298 non-null   float64
 19   D_72

### 3.  fillna with mean

In [464]:
for x in feature_name:
    feature[x]=feature[x].fillna(feature[x].mean())

### 4. split data in different category, prepare for stacking

In [465]:
delinquency=[]
spend = []
payment = []
balance = []
risk = []
std = []
mean = []
_25 = []
_50 = []
_75 = []
_min = []
_max = []

for x in feature_name:
    if x[0]=='D':
        delinquency.append(x)
    if x[0]=='S':
        spend.append(x)
    if x[0]=='P':
        payment.append(x)
    if x[0]=='B':
        balance.append(x)
    if x[0]=='R':
        risk.append(x)
    if x[-3:]=='std':
        std.append(x)
    elif x[-3:]=='25%':
        _25.append(x)
    elif x[-3:]=='75%':
        _75.append(x)
    elif x[-3:]=='50%':
        _50.append(x)
    elif x[-3:]=='min':
        _min.append(x)
    elif x[-3:]=='max':
        _max.append(x)
    else:
        mean.append(x)  

### 5. using GBC to check feature importance in original data

In [466]:
start_time = time.time()
GBC = GradientBoostingClassifier()
GBC.fit(feature,  np.array(label).T[0])
feature_importance = permutation_importance(GBC, feature, np.array(label).T[0], n_repeats=10)
print(" %s second " %(time.time()-start_time))

 3104.9427444934845 second 


In [467]:
fi = pd.DataFrame(feature_importance.importances_mean, index = feature.columns, columns = ['importance'])
important_l1= feature[fi.loc[fi['importance']>0].index]
unimportant_l1 = feature[fi.loc[fi['importance']==0].index]

### 6. creating input for stacking function

In [468]:
feature_sets = {'delinquency':delinquency,'spend':spend,'payment':payment,'balance':balance,'risk':risk,'std':std,
                'mean':mean,'_25':_25,'_50':_50,'_75':_75,'_min':_min,'_max':_max, 'unimportant_fe':unimportant_fe.columns}

### 7. stack feature by using level 1 prediction

In [498]:
model_dic ={}
for x in models:
    model_dic[x.__name__] = x

In [470]:
#GGLGRGLLLLLL
training_stacker(feature_sets)

-- delinquency GradientBoostingClassifier
-- spend GradientBoostingClassifier
-- payment LogisticRegression
-- balance RandomForestClassifier
-- risk RandomForestClassifier
-- std RandomForestClassifier
-- mean LogisticRegression
-- _25 GradientBoostingClassifier
-- _50 GradientBoostingClassifier
-- _75 LogisticRegression
-- _min GradientBoostingClassifier
-- _max LogisticRegression
-- unimportant_fe GradientBoostingClassifier


In [497]:
def stacker(feature,feature_sets):
    column_name = list(feature_sets.keys())
    r = pd.read_csv('model_for_stacking_l1.csv')   
    stack_feature = pd.DataFrame(columns = column_name)
    for x in column_name:
        print(x, end='-->')
        best_stacking_model = r[x].value_counts().keys()[0]
        stacking_feature = np.array(feature[feature_sets[x]])
        stack_feature[x] = level_1_predictor(stacking_feature,np_label.T[0],model_dic[best_stacking_model])
        print(best_stacking_model)
    return stack_feature

In [460]:
stacker(feature, feature_sets)

delinquency
spend
payment
balance
risk
std
mean
_25
_50
_75
_min
_max
unimportant_fe


,delinquency,spend,payment,balance,risk,std,mean,_25,_50,_75,_min,_max,unimportant_fe
0,[0.016632950446123153],[0.04327253174261714],[0.021504334159336103],[0.06],[0.05],[0.03],[0.005503991045609579],[0.013849236292664666],[0.018757194540519412],[0.0047416946089244546],[0.017754467484485253],[0.01],[0.01]
1,[0.86182861049515],[0.3905241825002649],[0.48094682687625134],[0.18],[0.71],[0.6],[0.7432397361076212],[0.7481276707146154],[0.8549063415136636],[0.8637063453595034],[0.40949067548631557],[0.56],[0.61]
2,[0.13261899804130556],[0.6450667239663932],[0.2422442569415946],[0.58],[0.3],[0.5],[0.5332516447850498],[0.341685011760477],[0.5662648869957428],[0.5530188955169997],[0.3410533954000603],[0.47],[0.36]
3,[0.09202205161392835],[0.028050275221921044],[0.06174617337574505],[0.06],[0.09],[0.0],[0.013238023892876353],[0.027593535839595576],[0.027578933886747436],[0.01941245079762113],[0.014717433858593684],[0.03],[0.04]
4,[0.04775181147619086],[0.03975612913180761],[0.05297884709229362],[0.03],[0.0],[0.04],[0.014467787840508627],[0.0248811716415156],[0.016402755455343636],[0.019330793022733078],[0.027604883908759294],[0.0],[0.05]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8293,[0.02000063026671058],[0.30932542243783634],[0.021048163572528592],[0.33],[0.55],[0.49],[0.00331379205910052],[0.019865799275810603],[0.013839902921998878],[0.0061304158657837626],[0.055142660408975044],[0.35],[0.34]
8294,[0.029971158268386294],[0.777256420902554],[0.14919346011844686],[0.36],[0.69],[0.49],[0.03657836346195438],[0.04538899322650132],[0.06086129442784356],[0.062451000962258495],[0.16494588225503137],[0.35],[0.37]
8295,[0.18674457141844414],[0.7571028259463505],[0.4032327888972332],[0.48],[0.55],[0.49],[0.12576620384984064],[0.515279263815979],[0.6885721730504502],[0.3243759162358339],[0.16201523291852007],[0.43],[0.48]
8296,[0.08654444709711724],[0.8081749140848145],[0.23315721051592053],[0.2],[0.58],[0.49],[0.05165800647501867],[0.0338071987896059],[0.19548777990534363],[0.07342429537480147],[0.2039508223992052],[0.23],[0.34]


In [409]:
o = pd.read_csv('model_for_stacking_l1.csv')

In [477]:
i = pd.DataFrame(dic)

In [ ]:
pd.to_csv()

### 8.  combine important_feature with stack feature. and select model to make final prediction

In [303]:
models = [GradientBoostingClassifier,RandomForestClassifier,LogisticRegression]

In [304]:
new_feature = pd.concat([important_fe,stack_features],axis=1)

In [305]:
X_train, X_test, y_train, y_test = train_test_split(new_feature,label, test_size=0.2, random_state=11)
for x in models:
    print(x.__name__,cross_validation(5,X_train, y_train, roc_auc_score,x))

. . . . . GradientBoostingClassifier 0.9428
. . . . . RandomForestClassifier 0.9428
. . . . . LogisticRegression 0.9439


### 9. check feature importance again using logist regression

In [306]:
LR = LogisticRegression(max_iter = 20**9)
LR.fit(new_feature,  np.array(label).T[0])
feature_importance_LR = permutation_importance(LR, new_feature, np.array(label).T[0], n_repeats=10)

In [307]:
fi_LR = pd.DataFrame(feature_importance_LR.importances_mean, index = new_feature.columns, columns = ['importance'])

In [472]:
fi_LR.loc[fi_LR['importance']>0]

,importance
P_2_mean,0.006483
D_39_mean,0.000048
D_43_mean,0.000976
D_51_mean,0.001398
S_5_mean,0.000012
...,...
_50,0.002314
_75,0.001579
_min,0.000759
_max,0.000410


In [308]:
important_feature_LR = new_feature[fi_LR.loc[fi_LR['importance']>0].index]
unimportant_feature_LR = new_feature[fi_LR.loc[fi_LR['importance']<=0].index]

### 10. created final_feature by using unimportant_feature to produce a new stacking feature, and add it on important_feature_LR

In [309]:
# use gradientBoosting to make level 1 prediction produces better result instead of Logist
unimportant_feature_LR_st = pd.DataFrame(level_1_predictor(unimportant_feature_LR,np_label.T[0], GradientBoostingClassifier),columns=['unimportant_feature_LR'])

In [310]:
final_feature = pd.concat([important_feature_LR,unimportant_feature_LR_st], axis=1)

### 11.  Final test

In [311]:
X_train, X_test, y_train, y_test = train_test_split(final_feature,label, test_size=0.2, random_state=11)
cross_validation(5,X_train, y_train, roc_auc_score,LogisticRegression)

. . . . . 

0.9471

----------

In [484]:
# create important_l1 file
dic = {}
for x in feature.columns:
    dic[x]=[0]
i = pd.DataFrame(dic)
i.to_csv('important_l1.csv', index=False)

In [486]:
i = pd.read_csv('important_l1.csv')
for x in important_l1:
    i[x]+=1
i.to_csv('important_l1.csv',index=False)

### experiments and draft code

In [495]:
# do run it ... unless you need it
def formating_model_selection():
    model_for_stacking_file = pd.DataFrame(columns = feature_sets.keys())
    model_for_stacking_file.to_csv('model_for_stacking_l1.csv', index = False)


In [294]:
df = pd.read_csv(r'C:\Users\johnk\OneDrive\Desktop\project\python project\kaggle\amex-default-prediction\new_train\train_1.csv')

In [59]:
X_train, X_test, y_train, y_test = train_test_split(feature,label, test_size=0.2, random_state=11)

In [90]:
feature_sets = {'delinquency':delinquency,'spend':spend,'payment':payment,'balance':balance,'risk':risk,'std':std,'mean':mean,'_25':_25,'_50':_50,'_75':_75,'_min':_min,'_max':_max}

In [32]:
stack_feature = pd.DataFrame(columns = [x[0] for x in feature_sets])

In [92]:
column_name = feature_sets.keys()
stack_feature = pd.DataFrame(columns = column_name)

In [106]:
np_label = np.array(label)

In [108]:
stack_features = stacker(feature_sets)

-- delinquency 

KeyboardInterrupt: 

In [261]:
X_train, X_test, y_train, y_test = train_test_split(stack_feature,label, test_size=0.2, random_state=11)
for x in models:
    print(x.__name__,cross_validation(5,X_train, y_train, roc_auc_score,x))

. . . . . GradientBoostingClassifier 0.941
. . . . . RandomForestClassifier 0.9395
. . . . . LogisticRegression 0.9444
. . . . . AdaBoostClassifier 0.9385


----

In [36]:
from sklearn.inspection import permutation_importance

In [104]:
np.array(label).T[0].shape

(8294,)

In [53]:
start_time = time.time()
LR = LogisticRegression(max_iter = 20**9)
LR.fit(feature,  np.array(label).T[0])
feature_importance_LR = permutation_importance(LR, feature, np.array(label).T[0], n_repeats=10)
print(" %s second " %(time.time()-start_time))

 476.11149883270264 second 


In [55]:
fi_LR = pd.DataFrame(feature_importance_LR.importances_mean, index = feature.columns, columns = ['importance'])

In [70]:
unimportant_feature = feature[fi.loc[fi['importance']==0].index]

In [63]:
important_feature_LR = feature[fi_LR.loc[fi_LR['importance']>0].index]

In [65]:
unimportant_feature_LR = feature[fi_LR.loc[fi_LR['importance']<=0].index]

In [71]:
X_train, X_test, y_train, y_test = train_test_split(unimportant_feature,label, test_size=0.2, random_state=11)
for x in models:
    print(x.__name__,cross_validation(5,X_train, y_train, roc_auc_score,x))

. . . . . GradientBoostingClassifier 0.9359
. . . . . RandomForestClassifier 0.9333
. . . . . LogisticRegression 0.9327
. . . . . AdaBoostClassifier 0.9246


In [179]:
start_time = time.time()
X_train, X_test, y_train, y_test = train_test_split(important_feature,label, test_size=0.2, random_state=11)
for x in [GradientBoostingClassifier]:
    print(x.__name__,cross_validation(5,X_train, y_train, roc_auc_score,x))
print(" %s second " %str(time.time()-start_time))

. . . . . GradientBoostingClassifier 0.9446
 402.04399275779724 second 


In [148]:
feature_sets_U ={'delinquency':delinquency,'spend':spend,'payment':payment,'balance':balance,'risk':risk,'std':std,'mean':mean, '_25':_25, '_50':_50, '_75':_75,'_min':_min,'_max':_max,'unimportant_feature':unimportant_feature.columns}

In [150]:
stack_features

,delinquency,spend,payment,balance,risk,std,mean,_25,_50,_75,_min,_max,unimportant_feature
0,0.015938,0.027792,0.016203,0.022692,0.08,0.019701,0.003754,0.003079,0.007215,0.006466,0.012077,0.001972,0.014232
1,0.062225,0.318739,0.061375,0.122451,0.19,0.054084,0.042429,0.044031,0.058534,0.045570,0.021609,0.025803,0.073997
2,0.309165,0.262442,0.341835,0.346352,0.09,0.074066,0.256343,0.287279,0.354750,0.287144,0.125587,0.239779,0.151567
3,0.911837,0.557271,0.956018,0.383776,0.79,0.844007,0.962661,0.966297,0.919914,0.863516,0.918122,0.731201,0.901779
4,0.816611,0.236555,0.475461,0.792159,0.02,0.492781,0.654506,0.528607,0.691080,0.663139,0.706979,0.808989,0.569937
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8289,0.561554,0.936307,0.563349,0.832050,0.62,0.314108,0.583158,0.139907,0.549369,0.533877,0.073816,0.266566,0.871721
8290,0.639513,0.745579,0.534681,0.690786,0.53,0.314108,0.102714,0.183770,0.294583,0.242028,0.102661,0.054970,0.533807
8291,0.069731,0.832446,0.243941,0.326528,0.67,0.314108,0.115424,0.060267,0.147698,0.129359,0.052932,0.060983,0.291663
8292,0.125865,0.875322,0.401100,0.464652,0.61,0.314108,0.241345,0.160496,0.302165,0.196338,0.171433,0.088653,0.323215


----

In [154]:
models = [GradientBoostingClassifier,RandomForestClassifier,LogisticRegression]

In [198]:
new_feature = pd.concat([important_feature,stack_features, unimportant_feature],axis=1)

In [199]:
X_train, X_test, y_train, y_test = train_test_split(new_feature,label, test_size=0.2, random_state=11)
for x in models:
    print(x.__name__,cross_validation(5,X_train, y_train, roc_auc_score,x))

. . . . . GradientBoostingClassifier 0.9437
. . . . . RandomForestClassifier 0.9423
. . . . . LogisticRegression 0.9362


In [185]:
LR = LogisticRegression(max_iter = 20**9)
LR.fit(new_feature,  np.array(label).T[0])
feature_importance_LR = permutation_importance(LR, new_feature, np.array(label).T[0], n_repeats=10)

In [186]:
fi_LR = pd.DataFrame(feature_importance_LR.importances_mean, index = new_feature.columns, columns = ['importance'])

In [187]:
better_feature = new_feature[fi_LR.loc[fi_LR['importance']>0].index]

In [188]:
X_train, X_test, y_train, y_test = train_test_split(better_feature,label, test_size=0.2, random_state=11)
for x in models:
    print(x.__name__,cross_validation(5,X_train, y_train, roc_auc_score,x))

. . . . . GradientBoostingClassifier 0.945
. . . . . RandomForestClassifier 0.9445
. . . . . LogisticRegression 0.9467


--------

In [189]:
fi_LR.loc[fi_LR['importance']>0]

,importance
P_2_mean,0.000109
S_3_mean,0.000060
B_4_mean,0.000313
D_48_mean,0.000217
B_11_mean,0.000543
...,...
mean,0.000916
_25,0.003834
_75,0.003617
_max,0.000916
